In [1]:
#pip install tika
#pip install inflect
import pandas as pd
import tika
from tika import parser
import re, string, unicodedata
import inflect
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
import re, math
from collections import Counter
from os import listdir
from os.path import isfile, join
# Download the package from nltk
# nltk.download()

In [2]:
import os
os.chdir('/Users/boyang/Desktop/Resume_samples')
os.getcwd()

'/Users/boyang/Desktop/Resume_samples'

In [3]:
# Helper function for text cleaning
# def remove_non_ascii(words):
#     """Remove non-ASCII characters from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#         new_words.append(new_word)
#     return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    date_sym = ["/","-"]
    for word in words:
        if word not in date_sym:
            new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(word_list):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_list = []
    for word in word_list:
        new_word = ""
        for char in word: 
            if char.isdigit():
                new_char = p.number_to_words(char)
                new_word = new_word + new_char
            else: 
                new_word = new_word + char
        new_list.append(new_word)
    return (new_list)

def stem_words(words):
    ps =PorterStemmer()
    return [ps.stem(w) for w in words]

def lemmatize_words(words):
    lemmatizer = WordNetLemmatizer() 
    return (lemmatizer.lemmatize(w,pos="a") for w in words)

def remove_stop_words(words):
    stop_words = set(stopwords.words('english')) 
    return [w for w in words if not w in stop_words] 

In [13]:
# Clean the data set
def cleaning_text(filename):
    parsed = parser.from_file(filename)
    content= parsed["content"].strip()
    word_list = nltk.word_tokenize(content)
    word_list = to_lowercase(word_list)
    word_list = remove_punctuation(word_list)
    word_list = replace_numbers(word_list)
    word_list = lemmatize_words(word_list)
    word_list = stem_words(word_list)
    word_list = remove_stop_words(word_list)
    word_text = ' '.join(word_list)
    
    return word_text

In [15]:
# cleaning_text("Yuanta Securities_Roy Li_Investment Analyst.pdf")
# cleaning_text("Xander_Rohit Khandelwal_VP.pdf")

In [5]:
# Cosine Similarity function
def text_to_vector(text):
    WORD = re.compile(r'\w+')
    words = WORD.findall(text)
    return Counter(words)

def get_cosine(text1, text2):
    
    vec1 = text_to_vector(text1)
    vec2 = text_to_vector(text2)
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [6]:
# Get filename
def resume_auto_scoring(job_description_file,resume_location):

    my_path = resume_location
    onlyfiles = [f for f in listdir(my_path) if isfile(join(my_path, f))]
    # Wierd value
    onlyfiles.remove('.DS_Store')
    # Remove the description file in the folder
    onlyfiles.remove(job_description_file)

    job_descrp = cleaning_text(job_description_file)
    similarity_score= []
    # Scan through all resumes and give a score ratings
    for file in onlyfiles:
        resume = cleaning_text(file)
        cosine = get_cosine(resume,job_descrp)
        similarity_score.append(cosine*100)
    df = {'filename': onlyfiles, 'similarity_score': similarity_score}
    df = pd.DataFrame(data=df)
    df = df.sort_values(by = "similarity_score",ascending = False)
    return df
    

In [7]:
# Job description filename: "Data_scientist_job_descrip.pdf"
# Resume path: "/Users/boyang/Desktop/Resume_samples"

job_descrip_file = "BA_Job_Descrip.pdf"
resume_path = "/Users/boyang/Desktop/Resume_samples"
score_df = resume_auto_scoring(job_descrip_file,resume_path)

In [12]:
score_df

,filename,similarity_score
935,IB_Laurens Brule_Biz Mgr.pdf,42.418193
839,GIC_Benjamin Chia_Business Analyst.pdf,40.943160
479,PWC_Osama Shah_M&A Advisory.pdf,39.350868
65,TES_Akash Shah_Group CFO.pdf,38.283603
201,UOB_Patcharapen_Client Acq Manager.pdf,38.168027
156,Citi_Yeo Hui Koon_Proj Mgr.pdf,38.129167
438,JANA_Sebastian Murphy_BD Exec.pdf,35.058835
15,Morningstar_Wenyin Xu_Insti Sales.pdf,33.596883
39,Citco_Jadey Li_BD.pdf,33.393884
358,Unicorn_Pratik Vora_Global Equity Analyst.pdf,33.220869
